In [1]:
import pandas as pd

import preprocessing as pp
import model
from train import train_model
from predict import predict

In [2]:
# Data loading
df_store = pd.read_csv('../data/store.csv')
df_train = pd.read_csv('../data/train.csv', low_memory=False)

## General pipeline test

### Model selection (Time-split cross-validation)

In [3]:
from sklearn.model_selection import TimeSeriesSplit
from train import rmse, r2

train = df_train.copy().iloc[::-1]
train.Date = pd.to_datetime(train.Date)

n_splits = 10
tscv = TimeSeriesSplit(n_splits=n_splits)

reg_model = model.Regressor()
rmse_scores = []
r2_scores = []

date_grouping = train.groupby(train.Date)['Store']
date_list = [g[0] for g in list(date_grouping)[:]]
for train_index, test_index in tscv.split(date_grouping):
    train_dates = [date_list[train_index[0]], date_list[train_index[-1]]]
    test_dates = [date_list[test_index[0]], date_list[test_index[-1]]]
    train_mask = (train.Date >= train_dates[0]) & (train.Date <= train_dates[1])
    test_mask = (train.Date >= test_dates[0]) & (train.Date <= test_dates[1])
    
    train.loc[test_mask]
    
    # Train and test sets
    X_train, y_train = pp.Preprocessor().transform(df_store, train.loc[train_mask])
    X_test, y_test = pp.Preprocessor().transform(df_store, train.loc[test_mask])
    
    # Dummy variables can induce differences in the schemas
    missing_test = set(X_train.columns) - set(X_test.columns)
    missing_train = set(X_test.columns) - set(X_train.columns)
    for c in missing_test:
        X_test[c] = 0
    for c in missing_train:
        X_train[c] = 0
    # Reorder to match columns order in train and test
    X_test = X_test[X_train.columns]
    
    # Model fitting on training set
    train_model(reg_model, X_train, y_train)

    # Scoring on test set
    y_pred = reg_model.predict(X_test)
    rmse_scores.append(rmse(y_test, y_pred))
    r2_scores.append(r2(y_test, y_pred))
        
# Final display
for i in range(n_splits):
    print("FOLD " + str(i + 1) + ": " + "RSME = " + str(rmse_scores[i]) + 
      " | R² = " + str(r2_scores[i]))

C:\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transf

FOLD 1: RSME = 2194.1681863962062 | R² = 0.6519610508937395
FOLD 2: RSME = 2189.6593190269905 | R² = 0.6277638280054414
FOLD 3: RSME = 2305.78621334217 | R² = 0.6143182864774158
FOLD 4: RSME = 2654.9290400425593 | R² = 0.5941517266817886
FOLD 5: RSME = 2181.4787727773355 | R² = 0.6806794715697491
FOLD 6: RSME = 2225.890826696642 | R² = 0.6591605044558997
FOLD 7: RSME = 2149.545245808583 | R² = 0.656797153365444
FOLD 8: RSME = 2678.091857956354 | R² = 0.598371018042982
FOLD 9: RSME = 2296.632440102387 | R² = 0.6533622396175565
FOLD 10: RSME = 2255.551092268223 | R² = 0.65731396715133


C:\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


### Model selection (Held-out test set of the last 6 weeks)

In [6]:
from sklearn.model_selection import TimeSeriesSplit
from train import rmse, r2

train = df_train.copy().iloc[::-1]
train.Date = pd.to_datetime(train.Date)
train_set = train[train.Date < '2015-06-19']
test_set = train[train.Date >= '2015-06-19']

reg_model = model.Regressor()
rmse_scores = []
r2_scores = []

X_train, y_train = pp.Preprocessor().transform(df_store, train_set)
X_test, y_test = pp.Preprocessor().transform(df_store, test_set)

# Dummy variables can induce differences in the schemas
missing_test = set(X_train.columns) - set(X_test.columns)
missing_train = set(X_test.columns) - set(X_train.columns)
for c in missing_test:
    X_test[c] = 0
for c in missing_train:
    X_train[c] = 0
# Reorder to match columns order in train and test
X_test = X_test[X_train.columns]

# Model fitting on training set
train_model(reg_model, X_train, y_train)

# Scoring on test set
y_pred = reg_model.predict(X_test)
rmse_scores = rmse(y_test, y_pred)
r2_scores = r2(y_test, y_pred)

print("RSME = " + str(rmse_scores) + " | R² = " + str(r2_scores))

RSME = 2253.766380366438 | R² = 0.6311535262941848


C:\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
